# Property Data Preprocessing and Cleaning

This notebook loads the raw `properti_data.csv`, standardizes the text, and then applies a series of specific, logical rules to clean the data and remove impossible outliers. It saves the final clean data and the removed outliers to separate files.

In [194]:
import pandas as pd
import re

## Step 1: Load and Standardize Raw Data
First, we load the raw dataset and standardize all text columns by removing quotes and converting to lowercase.

In [195]:
try:
    raw_df = pd.read_csv('properti_data.csv')
    display(raw_df.head())
except FileNotFoundError:
    print("Error: 'csv file not found. Make sure the file is in the same folder as this notebook.")

summary_df_part1 = pd.DataFrame({
    "Metric": ["Number of Rows", "Number of Columns", "Duplicate Rows"],
    "Value": [raw_df.shape[0], raw_df.shape[1], raw_df.duplicated().sum()]
})
display(summary_df_part1)

summary_df_part2 = pd.DataFrame({
    "Column": raw_df.columns,
    "Data Type": raw_df.dtypes.values,
    "Missing Values": raw_df.isnull().sum().values
})
display(summary_df_part2)

,Title,Address,Price,Bedrooms,Building Area (m²),Land Area (m²)
0,Dijual Rumah Posisi Hook dan Full Furnished di...,"Gading Serpong, Tangerang","Rp 2,30M",2.0,68m²,130m²
1,BELLEFONT EAST GADING SERPONG RUMAH 3 LANTAI G...,"Gading Serpong, Tangerang","Rp 4,20M",5.0,192m²,128m²
2,"RUMAH MEWAH FULL FURNISH HOOK, GADING SERPONG ...","Gading Serpong, Tangerang","Rp 2,80M",3.0,94m²,158m²
3,DIJUAL HARGA BAGUSS ALICANTE Lbr8m RAPIH HARGA...,"Alicante Gading Serpong, Tangerang","Rp 2,80M",3.0,125m²,144m²
4,Rumah di PHG Gading serpong hanya Jalan kaki 2...,"Gading Serpong, Tangerang","Rp 7,50M",4.0,325m²,270m²


,Metric,Value
0,Number of Rows,42322
1,Number of Columns,6
2,Duplicate Rows,2120


,Column,Data Type,Missing Values
0,Title,object,0
1,Address,object,0
2,Price,object,0
3,Bedrooms,float64,10
4,Building Area (m²),object,11
5,Land Area (m²),object,11


In [196]:
original_df = raw_df.copy() # Keep a copy of the original data for reference

for col in original_df.select_dtypes(include=['object']).columns:
    # Chain the string operations: first remove quotes, then convert to lowercase
    original_df[col] = original_df[col].str.replace('"', '', regex=False).str.lower()

df_clean = original_df.copy()
    
display(df_clean.head())

,Title,Address,Price,Bedrooms,Building Area (m²),Land Area (m²)
0,dijual rumah posisi hook dan full furnished di...,"gading serpong, tangerang","rp 2,30m",2.0,68m²,130m²
1,bellefont east gading serpong rumah 3 lantai g...,"gading serpong, tangerang","rp 4,20m",5.0,192m²,128m²
2,"rumah mewah full furnish hook, gading serpong ...","gading serpong, tangerang","rp 2,80m",3.0,94m²,158m²
3,dijual harga baguss alicante lbr8m rapih harga...,"alicante gading serpong, tangerang","rp 2,80m",3.0,125m²,144m²
4,rumah di phg gading serpong hanya jalan kaki 2...,"gading serpong, tangerang","rp 7,50m",4.0,325m²,270m²


## Step 2: Remove Duplicates and Missing Values

Before we do any complex cleaning, we'll perform two basic data hygiene steps:
1.  **Remove Duplicate Rows:** Any rows that are exact copies of another will be dropped.
2.  **Remove Rows with Missing Values:** Any rows with empty cells will be dropped.

In [197]:
print(f"Rows before cleaning: {len(df_clean)}")

# --- Drop Duplicate Rows ---
initial_rows = len(df_clean)
df_clean.drop_duplicates(inplace=True)
rows_dropped = initial_rows - len(df_clean)
if rows_dropped > 0:
    print(f"Removed {rows_dropped} duplicate rows.")

# --- Drop Rows with Missing Values ---
initial_rows = len(df_clean)
df_clean.dropna(inplace=True)
rows_dropped = initial_rows - len(df_clean)
if rows_dropped > 0:
    print(f"Removed {rows_dropped} rows with missing values.")

print(f"Rows after cleaning: {len(df_clean)}")

# Update the original_df to match this cleaned state for future reference
original_df = df_clean.copy()

Rows before cleaning: 42322
Removed 2144 duplicate rows.
Removed 14 rows with missing values.
Rows after cleaning: 40164


## Step 3: Convert Price Column
We define a function to convert price strings (e.g., 'rp 2,75m', 'rp 650jt') into a numerical format.

In [198]:
def convert_price(price_str):
    if not isinstance(price_str, str) or not re.search(r'\d', price_str): return None
    price_str = price_str.replace('rp', '').replace(' ', '').replace(',', '.')
    try:
        if 'm' in price_str: num = float(price_str.replace('m', '')) * 1_000_000_000
        elif 'jt' in price_str: num = float(price_str.replace('jt', '')) * 1_000_000
        else: num = float(price_str)
        return int(num)
    except ValueError: return None

df_clean['Price'] = df_clean['Price'].apply(convert_price)

rows_before = len(df_clean)
df_clean.dropna(subset=['Price'], inplace=True)
rows_after = len(df_clean)

print("Price column converted to numbers.")
if rows_before > rows_after:
    print(f"Removed {rows_before - rows_after} rows with unconvertible prices.")

display(df_clean[['Price']].head())

Price column converted to numbers.
Removed 11 rows with unconvertible prices.


,Price
0,2.300000e+09
1,4.200000e+09
2,2.800000e+09
3,2.800000e+09
4,7.500000e+09


## Step 4: Convert Feature Columns
We extract the numerical parts from the `Bedrooms`, `Building Area`, and `Land Area` columns.

In [199]:
for col in ['Bedrooms', 'Building Area (m²)', 'Land Area (m²)']:
    df_clean[col] = df_clean[col].astype(str).str.extract(r'(\d+)').astype(float)

print("Feature columns converted to numbers.")
display(df_clean[['Bedrooms', 'Building Area (m²)', 'Land Area (m²)']].head())

Feature columns converted to numbers.


,Bedrooms,Building Area (m²),Land Area (m²)
0,2.0,68.0,130.0
1,5.0,192.0,128.0
2,3.0,94.0,158.0
3,3.0,125.0,144.0
4,4.0,325.0,270.0


## Step 5: Outlier Detection and Flagging
We apply our specific logical rules to identify and flag outliers, adding the reason for rejection to a new column.

In [200]:
df_clean['rejection_reason'] = ''

# Rule 1: Impossible Bedroom Density
bed_density_mask = df_clean['Bedrooms'] / df_clean['Building Area (m²)'] >= 0.1
df_clean.loc[bed_density_mask, 'rejection_reason'] += 'Impossible bedroom density; '

# Rule 2: Unrealistically low price for land area
price_land_mask = (df_clean['Price'] / df_clean['Land Area (m²)']) < 700_000
df_clean.loc[price_land_mask, 'rejection_reason'] += 'Unrealistically low price for land area; '

# Rule 3: Unrealistically high price for building area
price_building_mask = (df_clean['Price'] / df_clean['Building Area (m²)']) > 150_000_000
df_clean.loc[price_building_mask, 'rejection_reason'] += 'Unrealistically high price for building area; '

# Rule 4: Impossible Building-to-Land Ratio
building_ratio_mask = (df_clean['Building Area (m²)'] / df_clean['Land Area (m²)']) > 10
df_clean.loc[building_ratio_mask, 'rejection_reason'] += 'Impossible building-to-land ratio; '

print("Outlier flagging complete.")
# Show a sample of rows that were flagged as outliers
display(df_clean[df_clean['rejection_reason'] != ''].head())

Outlier flagging complete.


,Title,Address,Price,Bedrooms,Building Area (m²),Land Area (m²),rejection_reason
694,rumah mewah ready stock new alicante tanpa dp,"gading serpong, tangerang",5.810000e+09,54.0,188.0,250.0,Impossible bedroom density;
1245,"dijual rumah shm 1 lantai di sektor 7a, gading...","gading serpong, tangerang",1.950000e+09,3.0,1.0,162.0,Impossible bedroom density; Unrealistically hi...
1753,rumah siap huni + garasi dan lokasi strategis ...,"cikupa, tangerang",6.500000e+08,3.0,1.0,90.0,Impossible bedroom density; Unrealistically hi...
1842,beli rumah dapat motor tanpa di undi promo 2.5...,"cikupa, tangerang",1.850000e+08,2.0,185000000.0,60.0,Impossible building-to-land ratio;
1984,dijual rumah di tangerang suvarna sutera clus...,"cikupa, tangerang",2.850000e+09,4.0,2165.0,180.0,Impossible building-to-land ratio;


## Step 6: Separate Clean Data from Outliers
Now, we split the DataFrame into two: one with the clean data and one with the rejected outliers.

In [201]:
cleaned_df_intermediate = df_clean[df_clean['rejection_reason'] == ''].copy()
dropped_df_intermediate = df_clean[df_clean['rejection_reason'] != ''].copy()

# Get original data for dropped rows for better context
dropped_df = original_df.loc[dropped_df_intermediate.index].copy()
dropped_df['rejection_reason'] = dropped_df_intermediate['rejection_reason']

print(f"Separation complete. Kept {len(cleaned_df_intermediate)} rows and identified {len(dropped_df)} outliers.")

Separation complete. Kept 39888 rows and identified 265 outliers.


## Step 7: Finalize the Clean DataFrame
We perform the final feature engineering (extracting the city) and select/reorder the final columns for our model-ready dataset.

In [202]:
def extract_city(address_str):
    if not isinstance(address_str, str): return 'other'
    # The address_lower is already done in the initial step
    if 'jakarta' in address_str: return 'jakarta'
    if 'tangerang' in address_str: return 'tangerang'
    if 'bekasi' in address_str: return 'bekasi'
    if 'depok' in address_str: return 'depok'
    if 'bogor' in address_str: return 'bogor'
    return 'other'

cleaned_df_intermediate.loc[:, 'City'] = original_df['Address'].loc[cleaned_df_intermediate.index].apply(extract_city)
cleaned_df = cleaned_df_intermediate.drop(columns=['Title', 'Address', 'rejection_reason'], errors='ignore')

for col in ['Price', 'Bedrooms', 'Building Area (m²)', 'Land Area (m²)']:
     cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce').astype('Int64')

final_cols = ['City', 'Bedrooms', 'Building Area (m²)', 'Land Area (m²)', 'Price']
cleaned_df = cleaned_df[final_cols]

display(cleaned_df.head())

,City,Bedrooms,Building Area (m²),Land Area (m²),Price
0,tangerang,2,68,130,2300000000
1,tangerang,5,192,128,4200000000
2,tangerang,3,94,158,2800000000
3,tangerang,3,125,144,2800000000
4,tangerang,4,325,270,7500000000


## Step 8: Save the Results
Finally, we save both DataFrames to new CSV files.

In [203]:
# Save the clean data
cleaned_df.to_csv('properti_data_cleaned.csv', index=False)
print("Cleaned data saved to 'properti_data_cleaned.csv'.")

# Save the dropped data
if not dropped_df.empty:
    dropped_df.to_csv('dropped_data.csv', index=False)
    print("Dropped outliers saved to 'dropped_data.csv'.")

Cleaned data saved to 'properti_data_cleaned.csv'.
Dropped outliers saved to 'dropped_data.csv'.
